<a href="https://colab.research.google.com/github/DanRivaille/Perceptron-Multicapa/blob/master/src/notebooks/NeuronalNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definición de la neurona
Se define la estructura de la neurona, la cual contendrá toda la información necesaria para que pueda realizar las operaciones que requiere.

In [20]:
import random

class Neuron:
  def __init__(self, function, value=0.0, id=None):
    self.value = value
    self.id = id
    self.funct = function
    self.der_funct = lambda x: self.funct(x) * (1 - self.funct(x))
    self.next_layer = []
    self.prev_layer = []
    self.curr_weights = []
    self.upd_weights = []
    self.bias = 0


  def add_layer(self, weights=None, bias=1, nodes_layer=None):
    self.prev_layer = nodes_layer
    cant_nodes = len(nodes_layer)
      
    if weights != None:
      self.curr_weights = weights
    else:
      self.curr_weights = [random.random() for i in range(cant_nodes)]

    self.bias = bias


  def calculate_value(self):
    sum = 0.0
    for (index, node) in enumerate(self.prev_layer):
      sum += node.value * self.curr_weights[index]

    self.value = self.funct(sum + self.bias)
    return self.value

## Probando la neurona
Se realizan algunas pruebas para verificar que todas las funciones las realiza de una forma correcta.
  Se crea una neurona que use la función sigmoide como función de activación, luego se le agrega una capa a partir de los valores que deberian tener los nodos. Por ultimo se calcula el valor activado de la neurona.

In [21]:
import math

def sigmoide(x):
  return 1 / (1 + math.exp(-x))

values = [0.34, 0.99, 0.03]
nodes = [Neuron(sigmoide, v) for v in values]

weights = [0.5, 4.7, 40.07]
biases = [0.1, 0.05, 0.08]

n = Neuron(sigmoide)
n.add_layer(weights, sum(biases), nodes)

n.calculate_value()

0.9980830478801297

# Definición de la red neuronal
Se defina la estructura de la red neuronal, la cual tendra las funciones básicas para que pueda operar.

In [22]:
class NeuronalNetwork:
  def __init__(self, cant_input, cant_output, function):
    self.input_layer = [Neuron(function, id=(0, i + 1)) for i in range(cant_input)]
    self.output_layer = [Neuron(function, id=(-1, i + 1)) for i in range(cant_output)]
    self.hidden_layers = []

  def is_empty(self):
    return len(self.hidden_layers) == 0

  def get_function(self):
    return self.input_layer[0].funct

  def add_layers(self, layers):
    # Se obtiene la malla
    self.hidden_layers = self._make_connections(layers)

    wi = [[0.1, 0.2, 0.15], [0.5, 0.01, 0.1]]
    wo = [[0.5, 0.6, 0.4], [0.3, 0.7, 0.1]]
    bi = [[0.2, 0.3, 0.01], [0.05, 0.03, 0.1]]
    bo = [[0.07, 0.1, 0.03], [0.009, 0.12, 0.023]]


    j = 0
    # Se conecta el principio de la malla con la capa de entrada
    for node in self.hidden_layers[0]:
      node.add_layer(nodes_layer=self.input_layer, weights=wi[j], bias=sum(bi[j]))
      j += 1

    j = 0
    # Se conecta la capa de salida con el final de la malla
    for node in self.output_layer:
      node.add_layer(nodes_layer=self.hidden_layers[-1], weights=wo[j], bias=sum(bo[j]))
      j += 1


  def predict(self, input):
    # Se copian los valores del input en los nodos de la capa de entrada
    for (index, value) in enumerate(input):
      self.input_layer[index].value = value

    # Se calculan los valores de las capas ocultas
    for layer in self.hidden_layers:
      for node in layer:
        node.calculate_value()
        print(f'{node.id}: {node.value}')

    # Se calcula la salida
    for node in self.output_layer:
      node.calculate_value()

    return [node.value for node in self.output_layer]


  def _make_connections(self, layers):
    '''
    Realiza las conexiones entre las capas ingresadas, y retorna una malla de 
    la red neuronal (sin entrada ni salida)
    '''
    nodes_mesh = self._create_mesh(layers)
    length = len(layers)

    # For debug
    ws = [[0.9, 0.8], [1.2], [0.7], [2.4]]
    bs = [[0.01, 0.02], [0.04], [0.05], [0.03]]

    j = 0

    for i in range(1, length):
      for node in nodes_mesh[i]:
        node.add_layer(nodes_layer=nodes_mesh[i - 1], weights=ws[j], bias=sum(bs[j]))
        j += 1

    return nodes_mesh


  def _create_mesh(self, layers):
    '''
    Crea una malla de capas, pero sin conecciones entre ellas
    '''
    nodes_mesh = []
    function = self.get_function()

    for current_layer in layers:
      new_nodes_layer = [Neuron(function, id=(current_layer, j + 1)) for j in range(current_layer)] # for debug
      #new_nodes_layer = [current_layer for j in range(current_layer)]  # for debug
      #new_nodes_layer = [Neuron(function) for j in range(current_layer)]
      nodes_mesh.append(new_nodes_layer)

    return nodes_mesh

In [24]:
layers = [2, 1, 3]

nn = NeuronalNetwork(3, 2, sigmoide)
nn.add_layers(layers)

In [23]:
# For debugging
def show_mesh(mesh):
  for layer in mesh:
    for node in layer:
      for (index, neight) in enumerate(node.prev_layer):
        print(f'{node.id} -> {neight.id}  w: {node.curr_weights[index]} v:{node.value}')

In [ ]:

d = 0.739304579579997 * 0.3 + 0.009 + 0.6535563064237191 * 0.7 + 0.12 + 0.8843939651086795 * 0.1 + 0.023
sigmoide(d)

0.7149850880113883

In [25]:
x = [3, 4.7, 7.8]
print(nn.predict(x))

(2, 1): 0.9488262990829084
(2, 2): 0.9246310913535561
(1, 1): 0.8352976939290254
(3, 1): 0.739304579579997
(3, 2): 0.6535563064237191
(3, 3): 0.8843939651086795
[0.7884393321603765, 0.7149850880113883]


In [27]:
print('cantidad de nodos de la capa anterior al primer nodo de la primera capa oculta:', len(nn.hidden_layers[0][0].prev_layer))

mesh = nn._make_connections(layers)
show_mesh(mesh)
print()

mesh = nn.hidden_layers
mesh.insert(0, nn.input_layer)
mesh.append(nn.output_layer)

print('cantidad de capas totales:', len(mesh))

show_mesh(mesh)
mesh

cantidad de nodos de la capa anterior al primer nodo de la primera capa oculta: 3
(1, 1) -> (2, 1)  w: 0.9 v:0.0
(1, 1) -> (2, 2)  w: 0.8 v:0.0
(3, 1) -> (1, 1)  w: 1.2 v:0.0
(3, 2) -> (1, 1)  w: 0.7 v:0.0
(3, 3) -> (1, 1)  w: 2.4 v:0.0

cantidad de capas totales: 5
(2, 1) -> (0, 1)  w: 0.1 v:0.9488262990829084
(2, 1) -> (0, 2)  w: 0.2 v:0.9488262990829084
(2, 1) -> (0, 3)  w: 0.15 v:0.9488262990829084
(2, 2) -> (0, 1)  w: 0.5 v:0.9246310913535561
(2, 2) -> (0, 2)  w: 0.01 v:0.9246310913535561
(2, 2) -> (0, 3)  w: 0.1 v:0.9246310913535561
(1, 1) -> (2, 1)  w: 0.9 v:0.8352976939290254
(1, 1) -> (2, 2)  w: 0.8 v:0.8352976939290254
(3, 1) -> (1, 1)  w: 1.2 v:0.739304579579997
(3, 2) -> (1, 1)  w: 0.7 v:0.6535563064237191
(3, 3) -> (1, 1)  w: 2.4 v:0.8843939651086795
(-1, 1) -> (3, 1)  w: 0.5 v:0.7884393321603765
(-1, 1) -> (3, 2)  w: 0.6 v:0.7884393321603765
(-1, 1) -> (3, 3)  w: 0.4 v:0.7884393321603765
(-1, 2) -> (3, 1)  w: 0.3 v:0.7149850880113883
(-1, 2) -> (3, 2)  w: 0.7 v:0.71498508

[[<__main__.Neuron at 0x7f1672945e90>,
 [<__main__.Neuron at 0x7f167236b250>, <__main__.Neuron at 0x7f167236b510>],
 [<__main__.Neuron at 0x7f167236b5d0>, <__main__.Neuron at 0x7f167236b8d0>]]